# Respiratory Signal Analysis

### Import packages

In [ ]:
import neurokit2 as nk
import os
import numpy as np
from scipy import signal
import csv
import matplotlib.pyplot as plt
from utils.load_data import load_csv_data_resp
%matplotlib inline
# %matplotlib tk

### Specify data path and initialize variables

In [ ]:
sampling_rate = 250.0
data_path = r'sample_data/'
filepath = os.path.join(data_path, 'P1_S1_baseline_1674758272_855629.csv')


### Load the data

In [ ]:
resp, event_code = load_csv_data_resp(filepath)

In [ ]:
time_axis = np.arange(len(resp))/sampling_rate
plt.plot(time_axis, resp, 'g')
plt.plot(time_axis[event_code != -1], resp[event_code != -1], 'ro', markersize=2)
plt.xlabel("Time (seconds)")
plt.ylabel("Signal Amplitude")


***
### Process Respiratory Signal
***

In [ ]:
sos = signal.butter(2, (0.1, 0.5), 'bandpass', fs=sampling_rate, output='sos')
filtered_resp = signal.sosfilt(sos, resp)

In [ ]:
# Clean signal
cleaned = nk.rsp_clean(filtered_resp, sampling_rate=sampling_rate)

# Extract peaks
df, peaks_dict = nk.rsp_peaks(cleaned) 
info = nk.rsp_fixpeaks(peaks_dict)
formatted = nk.signal_formatpeaks(info, desired_length=len(cleaned),peak_indices=info["RSP_Peaks"])

In [ ]:
fixed_peaks = nk.events_plot(info['RSP_Peaks'], cleaned)

In [ ]:
# Extract rate
rsp_rate = nk.rsp_rate(cleaned, peaks_dict, sampling_rate=sampling_rate)

# Visualize
nk.signal_plot(rsp_rate, sampling_rate=sampling_rate)
plt.ylabel('Breaths Per Minute')

In [ ]:
rrv = nk.rsp_rrv(rsp_rate, info, sampling_rate=sampling_rate, show=True)
rrv